In [ ]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime, openpyxl
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
import nltk
from nltk.corpus import stopwords 
sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

#df, code = generate_code_from_db(ctx, qgroup=4028)

In [ ]:
ldf = pd.read_csv(f'{ctx.get_dest()}ldf.csv').loc[:,['col','icn']] 
df=sql_todf("""
SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
LEFT JOIN PickListTypes ON UsageField=COLUMN_NAME
WHERE TABLE_NAME = 'PQCInventory'  
""", connstr)   
 
df = normalizedf(df)
df = pd.merge(df,ldf, left_on='COLUMN_NAME', right_on='col')
#df.loc[df['PK_PickListType'] != '0']
df


In [8]:

def get_datafield(df, out=False):
    txt=[]   
    for i,r in df.iterrows():
        plt=''
        PK_PickListType=r['PK_PickListType']
        COLUMN_NAME=r['COLUMN_NAME']
        icn=r['icn']
        DATA_TYPE=r['DATA_TYPE'] 
        if PK_PickListType != '0':  
            plt=f' PK_PickListType="{PK_PickListType}" '
        s=f'<CB:DataField DBColumnName="{COLUMN_NAME}" ImportColumnName="{icn}" {plt} runat="server"/>'
        
        txt.append(f'{s}')
    txt='\n'.join(txt) 
    with open(f'{ctx.get_dest()}script.aspx', 'w', encoding='UTF-8') as fw:
        fw.write(txt)
    if out: print(txt)

def get_params(df,out=False):
    parms=[]
    for i,r in df.iterrows():
        s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()}({r['CHARACTER_MAXIMUM_LENGTH']}) = NULL"
        if r['DATA_TYPE'] == 'INT':
            s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL" 
        parms.append(f',{s}')

    into=', '.join([c for c in df['COLUMN_NAME'] if 'PK_PQCInventory' not in c])
    vals=', @'.join([c for c in df['COLUMN_NAME'] if 'PK_PQCInventory' not in c]) 
    ins=f'\n\n \t\t \t\t INSERT INTO PQCInventory ({into})'
    ins=ins+f'\n \t\t \t\t VALUES (@{vals})'

    parms='\n\t\t'.join(parms)[:]
  
    update=', '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ]) 

    ext='\t\t \t\t AND ' + ' AND '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])

    sel,exp=[],[]
    for i,r in df.iterrows():
        s=f", {r['COLUMN_NAME']} "
        ex=f", {r['COLUMN_NAME']} AS [{r['icn']}]"
        if r['DATA_TYPE'] == 'INT':
            s=f", (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['COLUMN_NAME']}]" 
            ex=f", (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['icn']}]" 
        sel.append(f'{s}')
        exp.append(f'{ex}')
    sel='\t\t \t\t'+'\n \t\t \t\t'.join(sel)
    exp='\t\t \t\t'+'\n \t\t \t\t'.join(exp)
    with open(f'{ctx.get_dest()}script.sql', 'w', encoding='UTF-8') as fw:
        fw.write(parms)
 
    return {'parms':parms,'ins':ins, 'update':update, 'sel':sel, 'ext':ext , 'exp':exp  }

def get_gridcols(df,out=False):
    ldf=df.to_dict(orient='records') 
    txt,ddl,dv,sel=[],[],[],[] 
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        t=f.read() 
        for d in ldf: 
            col=d['col']
            s=t.replace("{col}",d['col']).replace("{ht}",d['icn'])
            sel.append(f',{col}')
            if d['PK_PickListType'] != '0':
                s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{col}" runat="server" />')
                ddl.append(f'BindDDL(e, "{col}", "{col}", "{col}")')
            else:
                s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{col}" runat="server" />')
                dv.append(f'DirectCast(e.Item.FindControl($"{col}"), RadTextBox).Text = drVal("{col}").ToString()')
            txt.append(f'{s}')     
    txt='\n'.join(txt) 
    ddl='\n'.join(ddl)
    dv= '\n'.join(dv)
    sel= '\n'.join(sel)[1:]
    with open(f'{ctx.get_dest()}script.aspx', 'w', encoding='UTF-8') as fw:
        fw.write(f'{txt} \n\n {ddl} \n\n {dv} \n\n {sel}')
    if out:  print(txt)
    return {'txt':txt, 'ddl':ddl,'dv':dv,'sel':sel}
  
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df['DATA_TYPE']=df['DATA_TYPE'].str.upper() 
    df['PK_PickListType']=df['PK_PickListType'].fillna(0).astype(int)
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].fillna(0).astype(int)
    df[cols] = df[cols].astype(str)
    df=df.loc[:, ['COLUMN_NAME','DATA_TYPE','CHARACTER_MAXIMUM_LENGTH','PK_PickListType']]
    return df
 
p = get_params(df, out=False)
print ( p['exp'] ) 
#get_datafield(df, out=True) 
#d=get_params(df, out=False)
#print(d['sel'])

		 		, AgencyName AS [Agency Name]
 		 		, SystemName AS [System Name]
 		 		, FISMASystemID AS [FISMA System ID]
 		 		, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist=FIPS199SystemCategorization) AS [FIPS 199 System Categorization]
 		 		, HighValueAssetID AS [High Value Asset ID]
 		 		, CryptoModuleName AS [Cryptographic Module Name]
 		 		, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist=CryptoAlgorithmUsed) AS [Cryptographic Algorithm Used]
 		 		, ServiceProvidedByCryptoModule AS [Service Provided By Cryptographic Module]
 		 		, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist=LengthCryptoKeysModules) AS [Length of associated cryptographic keys or modules]
 		 		, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist=SoftwarePackageType) AS [Software Package Type (COTS/GOTS/NDI/Other)]
 		 		, NameofVendorDev AS [Name of Vendor/Developer]
 		 		, SoftwarePackageNames AS [Software Package Name(s) (if commercial or open source); if unknown input 'N/A']
 

In [ ]:
d=get_params(df, out=False)
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\PQCInventory_CRUD.sql', 'r', encoding='UTF-8') as f:
    sql=f.read()
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\PQCInventory_CRUD.sql', 'w', encoding='UTF-8') as f:
    sql=re.sub('--parms(?s).*--~parms',f'--parms\n{d["parms"]}\n\t\t--~parms', sql)
    sql=re.sub('--sel(?s).*--~sel',f'--sel\n{d["sel"]}\n\t\t--~sel', sql)
    sql=re.sub('--ins(?s).*--~ins',f'--ins\n{d["ins"]}\n\t\t--~ins', sql)
    sql=re.sub('--update(?s).*--~update',f'--update\n{d["update"]}\n\t\t--~update', sql)
    sql=re.sub('--ext(?s).*--~ext',f'--ext\n{d["ext"]}\n\t\t--~ext', sql)
    f.write(sql)
 

In [ ]:
d=get_params(df, out=False)
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\PQCInventory_CRUD.sql', 'r', encoding='UTF-8') as f:
    sql=f.read()
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\PQCInventory_CRUD.sql', 'w', encoding='UTF-8') as f:
    sql=re.sub('--parms(?s).*--~parms',f'--parms\n{d["parms"]}\n--~parms\n', sql)
    sql=re.sub('--sel(?s).*--~sel',f'--sel\n{d["sel"]}\n--~sel\n', sql)
    sql=re.sub('--ins(?s).*--~ins',f'--ins\n{d["ins"]}\n--~ins\n', sql)
    sql=re.sub('--update(?s).*--~update',f'--update\n{d["update"]}\n--~update\n', sql)
    f.write(sql)
#print(sql)
    #

In [ ]:
d=get_gridcols(df, out=False)
print ( d['sel'] )

In [ ]:
d=get_gridcols(df, out=False)
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls\PQCInventory.ascx', 'r', encoding='UTF-8') as f:
    aspx=f.read()
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls\PQCInventory.ascx', 'w', encoding='UTF-8') as f:
    aspx=re.sub('<%-- cols --%>(?s).*<%-- ~cols --%>',f'<%-- cols --%>\n{d["txt"]}\n<%-- ~cols --%>\n', aspx)
    f.write(aspx)